In [2]:
from utils.classifier import *
from utils.dataloader import *
from utils.features import *
from utils.helper import *
from utils.processing import *
import pickle

In [3]:
RANDOM_STATE = 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
PARAMS = []
NUMB_LAEBLS = 8
L= 15
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [4]:
from sklearn.model_selection import train_test_split
img_path, prob, types, labels= load_data(DATA_PATH, NUMB_LAEBLS)

feature_builder = FeatureExtraction(DATA_PATH, img_path, labels, L)
X_train, X_test, y_train, y_test = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.25)
# X_train, X_val, y_train, y_val = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.1)
print(X_train.shape, X_test.shape)

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]
(1968, 300, 300) (656, 300, 300)


In [5]:
from sklearn.metrics import make_scorer, f1_score, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

preprocess_pipeline = [
    # (strech_img, {
    #     }),
    # (clach_img, {
    #     'clipLimit': 2.0,
    #     'tileGridSize': (8, 8)}),
    # (guassian_blur, {
    #     'kernel_size': (0, 0),
    #     'sigmaX': 3}),
    # (lap_feature, {
    #     'dst': -1,
    #     'ksize' : 5}),
    # (morpo_opening, {
    #     'kernel': np.ones((2,2), np.uint8),
    #     'iterations': 1}),
    (morph_smoothing, {
        'ses' : feature_builder.ses})
]

svm_param = {
    'C': 10,
    'kernel' : 'rbf',
    # 'gamma' : 0.0001,
    # 'class_weight' : 'balanced',
    # 'verbose' : False,
}

grid_svm_params = {
    'estimator': SVC,
    'model_params': {
        'kernel': ['rbf', 'linear', 'poly'],
        'C' : [0.001, 0.01, 0.1, 1, 10, 100],
        'class_weight': ['balanced']},
    'scoring': {
        "F1_Weighted": make_scorer(f1_score, average='weighted'),
        "Balanced_Accuracy": make_scorer(balanced_accuracy_score),},
    'refit': 'Balanced_Accuracy',
    'verbose' : 3
}

In [6]:
# X_train_prepro = feature_builder.preprocess(X_train, preprocess_pipeline = preprocess_pipeline)
# X_test_prepro = feature_builder.preprocess(X_test, preprocess_pipeline = preprocess_pipeline)

# with open(os.path.join(OUT_PATH ,'X_prepro.pkl'), 'wb') as f:
#     pickle.dump((X_train_prepro, X_test_prepro), f)

In [7]:
X_train_prepro,  X_test_prepro = None, None

with open(os.path.join(OUT_PATH ,'X_prepro.pkl'), 'rb') as f:
    X_train_prepro, X_test_prepro = pickle.load(f)

In [8]:
from sklearn.decomposition import FastICA

indices = np.where((y_train == 0) | (y_train == 4))[0]

non_defecet_X = X_train_prepro[indices]
non_defecet_y = y_train[indices]

ica = FastICA(random_state=RANDOM_STATE)
U = ica.fit_transform(X_train_prepro.reshape(X_train_prepro.shape[0], -1)) 
print(U.shape)

/opt/homebrew/Caskroom/miniconda/base/envs/torch/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


KeyboardInterrupt: 

In [11]:
U_plus = np.linalg.pinv(U)

b_train = np.dot(X_train_prepro.reshape(X_train_prepro.shape[0], -1), U_plus)
b_test = np.dot(X_test_prepro.reshape(X_test_prepro.shape[0], -1), U_plus)

ValueError: shapes (1968,90000) and (100,1968) not aligned: 90000 (dim 1) != 100 (dim 0)

In [ ]:
results, clf = grid_search(b_train, y_train, **grid_svm_params)
pred = clf.predict(b_test)

acc, conf_mat = get_report(y_test, pred)

print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)